In [28]:
from datetime import datetime, timedelta
import os
import openpyxl
import os
import pandas as pd

ROOT_DIR = './'

MARKS = ['СКС', "СКД", "ДСТ", "СБС","ДССК"]
print("Ready!")

Ready!


In [2]:
#Загрузка образца отчёта
wb = openpyxl.load_workbook('weekly_basis.xlsx')
report_sheet = wb['Форма горизонталь1']


report_titles = report_sheet['A1':'AL3']
hiden_data = report_sheet['A4':'G8']
rows = {'sks':report_sheet['G4':'AL4'],'skd':report_sheet['G5':'AL5'],'tep35':report_sheet['G6':'AL6'],
        'tep50':report_sheet['G7':'AL7'],'total':report_sheet['G8':'AL8']}



C:\ProgramData\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:300: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
C:\ProgramData\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:300: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [18]:
#Подгрузка отчётов из SAP 

#добавить проверку на верность даты начала и конца выгрузки
prod_report  = pd.read_excel(os.path.join(ROOT_DIR, 'ZPP_PROD_REPORT.XLSX'),sheet_name = 'Sheet1', thousands = ',').drop(columns='Продукт').rename(columns={'Продукт.1':'Продукт'})
prod_report

,Продукт,Партия,Партия(SAP),Начальная дата,Запас на начало,Производство,Перемаркировка,Паспортизация,Отгрузка,Запас на конец,Конечная дата
0,Продукт ЭП-ДССК А,9220-0002,0014804936,2020-09-21,2.003,0.000,0.0,0.000,0.000,2.003,2020-09-27
1,Продукт ЭП-ДССК А,9220-0010,0018377550,2020-09-21,16.704,0.873,0.0,0.000,0.000,17.577,2020-09-27
2,Продукт ЭП-ДССК А,9220-0011,0019365847,2020-09-21,9.273,1.431,0.0,0.000,0.000,10.704,2020-09-27
3,Продукт ЭП-ДССК А,9220-0013,0019786366,2020-09-21,6.146,12.978,0.0,0.000,0.000,19.124,2020-09-27
4,Продукт ЭП-ДССК А,9220-0014,0019974469,2020-09-21,0.000,4.814,0.0,0.000,0.000,4.814,2020-09-27
...,...,...,...,...,...,...,...,...,...,...,...
2252,ТЭП СБС Р30-00AN порошок мешок,4220N9029,0020258485,2020-09-21,0.000,0.000,0.0,0.000,0.000,0.000,2020-09-27
2253,ТЭП СБС Р30-00AN порошок мешок,4220N9029,ГП_0258485,2020-09-21,0.000,0.000,0.0,0.000,0.000,0.000,2020-09-27
2254,ТЭП СБС Р30-00К гр меш ТТ 00148889-026,4520S7007,0018300643,2020-09-21,5.448,0.000,0.0,0.000,0.000,5.448,2020-09-27
2255,NaN,NaN,NaN,NaT,440.056,219.400,-1.8,206.460,82.800,399.882,NaT


In [61]:
#Используемые функции(отлаженные и протестированные)
#печатает слайс
def print_row(row_slice):
    for cell_obj in row_slice:
        for cell in cell_obj:
            print(cell.value, cell.column, cell.coordinate)
#создает эксель, в который пишутся результаты
def create_resulting_wb():
    wb_result = openpyxl.Workbook()
    wb_result.create_sheet(title = 'Отчёт', index = 0)
    result_sheet = wb_result['Отчёт']
    for irow in range(8):
        result_sheet.append([])
    wb_result.save('weekly_report.xlsx')
    return wb_result
#копирую инфу из одного экселя в другой
def copy_cells(slice_to_copy,sheet_we_copy_in):
    for cells_obj in slice_to_copy:
        for cell in cells_obj:
            cell_tmp = sheet_we_copy_in.cell(row = cell.row, column = cell.column)
            cell_tmp.value = cell.value
    wb_result.save('weekly_report.xlsx')
#Посчитать последнюю строку    
def find_total():
    total_positions = []
    for cell_obj in rows['total']:
        for cell in cell_obj:
            if type(cell.value) == str:
                total_positions.append((cell.value, cell.column, cell.coordinate))
    for i in total_positions:
        try:
            report_sheet[i[2]] = (
                                  float(report_sheet.cell(column=i[1],row=4).value) + 
                                  float(report_sheet.cell(column=i[1],row=5).value) + 
                                  float(report_sheet.cell(column=i[1],row=6).value) + 
                                  float(report_sheet.cell(column=i[1],row=7).value)
                                  )
        except ValueError:
            #print(i[2])
            report_sheet[i[2]] = ''
    #return report_sheet['G8':'AL8']

def put_info_in_col(col_n, data_dict, sheet_we_put_in):
    cell_tmp = sheet_we_put_in.cell(row = 4, column = col_n)#CKC
    cell_tmp.value =data_dict['СКС']
    cell_tmp = sheet_we_put_in.cell(row = 5, column = col_n)#CKC
    cell_tmp.value =data_dict['СКД']
    cell_tmp = sheet_we_put_in.cell(row = 6, column = col_n)#CKC
    cell_tmp.value =data_dict['ДСТ']
    cell_tmp = sheet_we_put_in.cell(row = 7, column = col_n)#CKC
    cell_tmp.value =data_dict['СБС']
    wb_result.save('weekly_report.xlsx')


#Получаем данные для 17стобца из ZPP_PROD_REPORT -> вариант VSK_PERIOD_PRD (Отчёт ВСК производства и сортировки продукции. Паспортизация)
def get_column_17_info(df_zpp_prod_report):
    #['СКС', "СКД"+"ДССК", "ДСТ", "СБС"]
    RESULT = {'СКС':0,'СКД':0,'ДСТ':0,'СБС':0}
    for i in range(prod_report.shape[0]):
        row_tmp = prod_report.iloc[i,:]
        row_tmp['Продукт'] = str(row_tmp['Продукт'])
        if MARKS[0] in row_tmp['Продукт'] and not 'ЭП' in row_tmp['Продукт']:#CKC - верно
            RESULT['СКС'] += row_tmp['Паспортизация'].astype(float) 
            continue
        if MARKS[1] in row_tmp['Продукт'] and not 'ЭП' in row_tmp['Продукт']:#СКД - верно
            RESULT['СКД'] += row_tmp['Паспортизация'].astype(float)
            continue
        if MARKS[4] in row_tmp['Продукт'] and not 'ЭП' in row_tmp['Продукт']:#ДССК - верно
            RESULT['СКД'] += row_tmp['Паспортизация'].astype(float)
            continue
        if MARKS[2] in row_tmp['Продукт']:# and not 'ЭП' in row_tmp['Продукт']:#ДСТ
            RESULT['ДСТ'] += row_tmp['Паспортизация'].astype(float)
            continue
        if MARKS[3] in row_tmp['Продукт']:# and not 'ЭП' in row_tmp['Продукт']:#СБС
            RESULT['СБС'] += row_tmp['Паспортизация'].astype(float) 
            continue
    return RESULT

In [62]:
#Основной список вызов функций (main)
wb_result = create_resulting_wb()
result_sheet = wb_result['Отчёт']
copy_cells(report_titles, result_sheet)
copy_cells(hiden_data, result_sheet)
#17 столбец
put_info_in_col(17,get_column_17_info(prod_report),result_sheet)
#последняя строка
find_total()
copy_cells(rows['total'], result_sheet)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [54]:
#MARKS = ['СКС', "СКД", "ДСТ", "СБС","ДССК"]


In [56]:
x = get_column_17_info(prod_report)
x
#rail = ORION[(ORION['Метод отгрузки'] == 'ЖД -вагон') & (ORION['Тип объекта'] == 'РО')][[date,'Наименование   позиции']].dropna().reset_index().drop(columns = 'index')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


{'СКС': 1329.5099999999998,
 'СКД': 2389.3199999999993,
 'ДСТ': 546.492,
 'СБС': 456.65999999999997}

In [30]:
RESULT = [0,0]
for i in range(prod_report.shape[0]):
    row_tmp = prod_report.iloc[i,:]
    row_tmp['Продукт'] = str(row_tmp['Продукт'])
    if MARKS[1] in row_tmp['Продукт'] and not 'ЭП' in row_tmp['Продукт']:#СКД
        RESULT[1] += row_tmp['Паспортизация'].astype(float)
        print(row_tmp)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


Продукт               Каучук СКД A м/к
Партия                       0120A0893
Партия(SAP)                 0020015959
Начальная дата     2020-09-21 00:00:00
Запас на начало                      0
Производство                         0
Перемаркировка                   27.72
Паспортизация                        0
Отгрузка                             0
Запас на конец                   27.72
Конечная дата      2020-09-27 00:00:00
Name: 159, dtype: object
Продукт               Каучук СКД A м/к
Партия                       0120A0893
Партия(SAP)                 ГП_0015959
Начальная дата     2020-09-21 00:00:00
Запас на начало                      0
Производство                         0
Перемаркировка                       0
Паспортизация                        0
Отгрузка                             0
Запас на конец                       0
Конечная дата      2020-09-27 00:00:00
Name: 160, dtype: object
Продукт            Каучук СКД B г/к 0,54
Партия                         0120-0797
Партия(SAP

Name: 305, dtype: object
Продукт            Каучук СКД B п/к 0,54
Партия                         0120-1011
Партия(SAP)                   0020253583
Начальная дата       2020-09-21 00:00:00
Запас на начало                        0
Производство                           0
Перемаркировка                         0
Паспортизация                          0
Отгрузка                               0
Запас на конец                         0
Конечная дата        2020-09-27 00:00:00
Name: 306, dtype: object
Продукт            Каучук СКД B п/к 0,54
Партия                         0120-1012
Партия(SAP)                   0020256281
Начальная дата       2020-09-21 00:00:00
Запас на начало                        0
Производство                           0
Перемаркировка                         0
Паспортизация                          0
Отгрузка                               0
Запас на конец                         0
Конечная дата        2020-09-27 00:00:00
Name: 307, dtype: object
Продукт            Кауч

Name: 451, dtype: object
Продукт            Каучук СКД/BR 1203 Ti B м/к 1,26
Партия                                    0120-0935
Партия(SAP)                              ГП_0118913
Начальная дата                  2020-09-21 00:00:00
Запас на начало                                   0
Производство                                      0
Перемаркировка                                    0
Паспортизация                                     0
Отгрузка                                          0
Запас на конец                                    0
Конечная дата                   2020-09-27 00:00:00
Name: 452, dtype: object
Продукт            Каучук СКД/BR 1203 Ti B м/к 1,26
Партия                                    0120-0937
Партия(SAP)                              0020127051
Начальная дата                  2020-09-21 00:00:00
Запас на начало                                   0
Производство                                      0
Перемаркировка                                    0
Паспортизация 

Name: 602, dtype: object
Продукт            Каучук СКД-НД ВВ м/к
Партия                        0520-0101
Партия(SAP)                  ГП_9791419
Начальная дата      2020-09-21 00:00:00
Запас на начало                       0
Производство                          0
Перемаркировка                        0
Паспортизация                     52.92
Отгрузка                              0
Запас на конец                     12.6
Конечная дата       2020-09-27 00:00:00
Name: 603, dtype: object
Продукт            Каучук СКД-НД ВВ м/к
Партия                        0520-0102
Партия(SAP)                  0019791814
Начальная дата      2020-09-21 00:00:00
Запас на начало                    75.6
Производство                          0
Перемаркировка                    -31.5
Паспортизация                         0
Отгрузка                              0
Запас на конец                        0
Конечная дата       2020-09-27 00:00:00
Name: 604, dtype: object
Продукт            Каучук СКД-НД ВВ м/к
Парти

Name: 755, dtype: object
Продукт            Каучук СКД-НД/BR-1243 Nd B LP м/к 1,26
Партия                                          0220-0357
Партия(SAP)                                    0020101474
Начальная дата                        2020-09-21 00:00:00
Запас на начало                                         0
Производство                                            0
Перемаркировка                                          0
Паспортизация                                           0
Отгрузка                                                0
Запас на конец                                          0
Конечная дата                         2020-09-27 00:00:00
Name: 756, dtype: object
Продукт            Каучук СКД-НД/BR-1243 Nd B LP м/к 1,26
Партия                                          0220-0357
Партия(SAP)                                    ГП_0101474
Начальная дата                        2020-09-21 00:00:00
Запас на начало                                         0
Производство          

In [45]:
RESULT1 = [0,0,0]
for i in range(prod_report.shape[0]):
    row_tmp = prod_report.iloc[i,:]
    row_tmp['Продукт'] = str(row_tmp['Продукт'])
    if z in row_tmp['Продукт'] and not 'ЭП' in row_tmp['Продукт']:#ДССК
        RESULT1[2] += row_tmp['Паспортизация'].astype(float)
        print(row_tmp)        
RESULT1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


[0, 0, 0]

In [44]:
z = prod_report.iloc[1370,0].split(' ')[1]
z

'ДСТ'